In [4]:
from google.oauth2 import service_account
from google.cloud import kms
from cryptography.fernet import Fernet

import firebase_admin
from firebase_admin import auth, credentials, firestore


Global params

In [2]:
# SERVICE_ACCOUNT_FILE = '../firebase-adminsdk.json'
KMS_SERVICE_ACCOUNT_FILE = '../kms-admin.json'

project_id = 'wallet-login-45c1c'

KMS params

In [3]:
# Build the parent name from the project and location.
parent = f"projects/{project_id}/locations/global"

# Build the key ring name.
key_ring_name = f"{parent}/keyRings/gladius-key-ring"

# Build the key name.
key_name = f"{key_ring_name}/cryptoKeys/gladius-key"

print(key_name)

projects/wallet-login-45c1c/locations/global/keyRings/gladius-key-ring/cryptoKeys/gladius-key


#### Connect to KMS

In [5]:

# Create the credentials object from the service account file.
kms_credentials = service_account.Credentials.from_service_account_file(KMS_SERVICE_ACCOUNT_FILE)

# Create the client with the credentials.
kms_client  = kms.KeyManagementServiceClient(credentials=kms_credentials)

#### Connect to Firebase

In [ ]:
firebase_cred = credentials.Certificate('../firebase-adminsdk.json')
firebase_admin.initialize_app(firebase_cred)
db = firestore.client()

In [14]:
user_id = 'jjzEiaH1SqN92fMuQSapSbKuHVB2'  # Bob
user_ref = db.collection('users').document(user_id)
encrypted_private_key = user_ref.get().get('glc_transaction_private_key')
encrypted_encryption_key = user_ref.get().get('encrypted_encryption_key')


In [17]:
response = kms_client.decrypt(
    request={"name": key_name, "ciphertext": encrypted_encryption_key}
)
encryption_key = response.plaintext


In [19]:

cipher_suite = Fernet(encryption_key)
#encrypted_private_key = base64.b64decode(encrypted_private_key)  # If necessary

decrypted_private_key = cipher_suite.decrypt(encrypted_private_key)
decrypted_private_key

b'Bob'

In [21]:
# encrypted_private_key was actually encrypted user_name
# glc_transaction_private_key = f.encrypt(user_name.encode())